# Function and Methods Overloading in Python 3

> This notebook is a simple example of how to overload functions and methods in Python 3. The base idea is to create a function or method that can receive different types of arguments and return different results based on the type of the arguments.

> Most of this notebook's content was based on the article by **Pawel Dudziniski**, [Function and Method Overloading in Python](https://medium.com/@paweldudzinski/functions-and-methods-overloading-in-python-3-9409233af5d2).

Functions overloading in Python is not possible in the same way as in other languages like C++ or Java. In Python, you can't define two functions with the same name. However, you can use the `@singledispatch` decorator from the `functools` module to create a function that can receive different types of arguments and return different results based on the type of the arguments. This can save us from ugly _ifology_ or class type checking to execute different code that depends on an `isinstance` function result.

Let's consider a simple class that represents a vector with possibility of getting item by index (code Python 3.10 compliant):

In [1]:
class Vector:
    def __init__(self, components: list | None = None) -> None:
        self._components = components or []

    def __getitem__(self, index: int) -> list:
        return self._components[index]

We can create a `Vector` and we can get an item by index:

In [2]:
vector = Vector(components=[0, 1, 2, 3, 4])
vector[3]

3

But what about slicing?

In [3]:
vector_slice = vector[2:4]
print(vector_slice, type(vector_slice))

[2, 3] <class 'list'>


Since we implemented the `__getitem__` method, Python can figure how to get a slice from the **Vector** object. But our `vector_slice` is a **list**, but we want a slice of a **Vector** to be a **Vector** too. We can implement the `__getitem__` method to return a **Vector** object when a slice is requested:

In [4]:
from typing import Union


class Vector:
    def __init__(self, components: list | None = None) -> None:
        self._components = components

    def __getitem__(
        self,
        index: Union[int, slice],
    ) -> Union[list, Vector]:
        _cls = type(self)
        if isinstance(index, slice):
            return _cls(components=self._components[index])
        elif isinstance(index, int):
            return self._components[index]
        else:
            raise TypeError('list indices must be integers.')


We need to check if `index` is `slice` or `int` and if it is a `slice`, we need to return a new `Vector` object with the slice of the `self._vector` list. We can use the `isinstance` function to check the type of the `index` argument.

But that kind of code is unreadable and confusing, and lets agree that not very pythonic.

It seems to be a good use case to use function overloading (or method in this very case), so we caould have separate methods that handles different types of `index` parameter. Code refactoring to meet the <u>single responsability principle</u> is always a way to go.

## Function Overloading

The `@singledispatch` decorator allows us to create a function that can receive different types of arguments and return different results based on the type of the arguments. The `@singledispatch` decorator is a generic function dispatcher that allows you to register a function to be called for different types of objects.

In [5]:
from functools import singledispatch


class Dog:
    pass


class Cat:
    pass


class Horse:
    pass


@singledispatch
def make_sound(animal: object) -> str:
    NotImplementedError('Type is not supported')


@make_sound.register
def _(animal: Dog) -> str:
    return 'Bark'

@make_sound.register
def _(animal: Cat) -> str:
    return 'Meow'

@make_sound.register
def _(animal: Horse) -> str:
    return "Neigh"


In [6]:
dog = Dog()
cat = Cat()
horse = Horse()

print(make_sound(dog))
print(make_sound(cat))
print(make_sound(horse))

Bark
Meow
Neigh


We have three simple classes that represents animals. We want to write a function that will return a sound for a given animal. We could do it by checking instance of an animal parameter and returning proper string in one function. But this is a great use case to overload the `make_sound` function.

First we implement an interface and decorate it with a `@singledispatch` that lives in the `functools` module. The we write as much `_` functions as many cases we need to cover, and decorate each with `@<fun_name>.register`. The implementation for each individual function may vary significantly.

For function overloading we use a single `@singledispatch` decorator, for instance methods there's a `@singledispatchmethod` decorator. In `Vector` case we have a class with a dunder method to overload, so we will use the second approach.

In [23]:
from functools import singledispatchmethod


class Vector:
    def __init__(self, components: list | None = None) -> None:
        self._components = components or []

    @singledispatchmethod
    def __getitem__(self, index: object) -> object:
        raise NotImplementedError('Unsupported type')

    @__getitem__.register
    def _(self, index: int) -> int:
        return self._components[index]

    @__getitem__.register
    def _(self, index: slice) -> Vector:
        return Vector(components=self._components[index])

    def __str__(self) -> str:
        return ', '.join(str(ele) for ele in self._components)


In [24]:
vector = Vector(components=[0, 1, 2, 3, 4])
vector[3]

3

In [26]:
vector_slice = vector[2:4]
print(f"[{vector_slice}]", type(vector_slice))

[2, 3] <class '__main__.Vector'>
